In [ ]:
#from FoodRatingDataSet import FoodRatingDataset

SEQ_LEN = 5

#data_set = FoodRatingDataset()

#Test
from TestDataSet import TestDataSet

data_set = TestDataSet()

NUM_ING = data_set.NUM_ING
OTHER_FEATURES = data_set.OTHER_FEATURES
NUM_RECIPES = data_set.NUM_RECIPES

In [ ]:
from FoodRecommenderModelSequence import FoodRecommenderModelSequence
import tensorflow as tf
import keras

ids, ing, ofe = tf.keras.Input(shape=[SEQ_LEN]), tf.keras.Input(shape=[SEQ_LEN, NUM_ING]), tf.keras.Input(
    shape=[SEQ_LEN, OTHER_FEATURES])

x = FoodRecommenderModelSequence(recipe_count=NUM_RECIPES, seq_len=SEQ_LEN)((ids, ing, ofe))

mod = keras.Model((ids, ing, ofe), x)
mod.summary()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

In [ ]:
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime
import os.path

data = data_set.data()

mod = FoodRecommenderModelSequence(recipe_count=NUM_RECIPES, seq_len=SEQ_LEN)
mod.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])

board_file_name = datetime.now().strftime("%d%m%Y_%H%M%S")

checkpoints_dir = "checkpoints/checkpoint"


if os.path.exists(checkpoints_dir):
    print("load model:")
    board_file_name = f"{board_file_name}_loaded"
    mod.load_weights(checkpoints_dir)
else:
    print("new model:")

tensorboard = TensorBoard(log_dir=f"logs/recommend_{board_file_name}")
checkpoints = ModelCheckpoint(filepath=checkpoints_dir, save_weights_only=True)

#mod.fit(data, epochs=10, validation_split=0.3, callbacks=[tensorboard, checkpoints])
mod.fit(data, epochs=10, callbacks=[tensorboard, checkpoints])